In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import tensorflow as tf
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 500
epochs = 50
n_classes = 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
import tensorflow as tf
from urllib.parse import urlparse

In [5]:
import pandas as pd
import tensorflow as tf
from FeatureCreation import create_features
AUTOTUNE = tf.data.AUTOTUNE

# Define a function to preprocess the link
def preprocess_link(link):
    # Convert all letters to lowercase
    link = link.lower()
    link = link.replace("http://","")
    link = link.replace("https://","")
    link = link.replace("www.","")
    link = link.replace(" ","")
    link = link.strip()
    # Remove punctuation and special characters
    link = re.sub('[^A-Za-z0-9]+', '', link)
    return link

def steps_per_epoch(dataframe,batch_size):
    return len(dataframe)//batch_size

def process_train_data(df,batch_size):
    df_numeric = create_features(df)
    df['url'] = df['url'].apply(preprocess_link)
    df = df.drop_duplicates(subset=['url'])
    df = df.dropna().reset_index(drop=True)
    ds = tf.data.Dataset.from_tensor_slices((df['url'].values, df_numeric.values, df['type'].values))
    ds = ds.batch(batch_size)
    return ds

def vectorize_text(text,numerics, label, Vectorize_Layer):
    text = tf.expand_dims(text, -1)
    return Vectorize_Layer(text), numerics, label

def process_data(df,batch_size, Vectorize_Layer):
    ds = process_train_data(df,batch_size)
    ds = ds.map(lambda x, y, z: vectorize_text(x,y,x,Vectorize_Layer))
    ds = ds.cache().prefetch(buffer_size=AUTOTUNE)
    return ds

def process_links(links,batch_size, Vectorize_Layer):
    df = pd.Dataframe()
    df['url'] = links
    # here type is not provided as these links type is to be predicted, so we are keeping 0 just only for maintaining dimensionality
    df['type'] = 0
    return process_data(df,batch_size, Vectorize_Layer)
    


In [6]:
import pandas as pd
df = pd.DataFrame()
df['url']=["https://www.google.com/"]
df['type']=[0]
df

,url,type
0,https://www.google.com/,0


In [4]:
from FeatureCreation import create_features
df = create_features(df)
df

,url,type,use_of_ip,abnormal_url,count.,count-www,count@,count_dir,count_embed_domian,count-https,count-http,count%,count?,count-,count=,url_length,hostname_length,count-digits,count-letters,fd_length
0,https://www.google.com/,0,0,1,2,1,0,1,0,1,1,0,0,0,0,23,14,0,17,0


In [ ]:
# Typically, cyber attackers replace the domain name in a URL with an IP address to conceal the website's identity. This feature is designed to verify whether the URL includes an IP address or not.
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        return 1
    else:
        return 0

In [9]:
# check whether link is in proper formatting or not
def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        return 1
    else:
        return 0

In [ ]:
# Phishing or malware websites often incorporate multiple sub-domains into their URLs, with each sub-domain separated by a dot (.). URLs containing more than three dots (.) are more likely to be malicious sites.
def count_dot(url):
    count_dot = url.count('.')
    return count_dot

In [ ]:
# In general, most safe websites contain only one "www" in their URLs. This feature can aid in identifying malicious websites by detecting URLs that either lack "www" or contain more than one instance of it.
def count_www(url):
    url.count('www')
    return url.count('www')

In [ ]:
# The occurrence of the "@" symbol in a URL causes everything preceding it to be disregarded.
def count_atrate(url): 
    return url.count('@')

In [ ]:
# Websites that contain multiple directories within their URLs are typically considered suspicious.
def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

In [ ]:
# Examining the frequency of the " //" sequence within a URL can aid in identifying malicious URLs with multiple embedded domains.
def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

In [22]:
# Malicious URLs often avoid using HTTPS protocols as they typically require user credentials and provide a layer of security for online transactions. Therefore, the presence or absence of HTTPS protocol in a URL can be a crucial indicator in determining its safety.
def count_https(url):
    return url.count('https')

['Linkjoy', 'Switchy', 'Bitly', 'Rebrandly', 'Cutt.ly', 'Sniply', 'BL.INK', 'RocketLink', 'BioLinky', 'Geniuslink', 'Clickmeter', 'Capsulink', 'Linkpop', 'Short.io', 'TinyURL', 'Replug', 'ShortURL', 'Bio Link', 'ClickToTweet', 'PixelMe', 'LnnkIn', 'smartURL']


In [ ]:
# Typically, phishing or malicious websites contain multiple instances of HTTP in their URL, whereas safe websites only have one.
def count_http(url):
    return url.count('http')

In [ ]:
# As URLs cannot contain spaces, they are often replaced with symbols (%), which is known as URL encoding. Safe websites tend to have fewer spaces in their URLs, while malicious sites often have more, resulting in a higher number of % symbols.
def count_per(url):
    return url.count('%')

In [ ]:
# A symbol (?) in a URL indicates the presence of a query string, which contains data to be sent to the server. If a URL contains multiple instances of the symbol (?), it can be a sign of a suspicious URL.
def count_ques(url):
    return url.count('?')

In [ ]:
# To make a URL appear genuine, phishers and cybercriminals often add dashes (-) to the prefix or suffix of a brand name. For instance, they may create a URL like www.flipkart-india.com.
def count_hyphen(url):
    return url.count('-')

In [ ]:
# The presence of an equal sign (=) in a URL implies that variable values are being passed from one form page to another. This is considered risky since anyone can modify the values and alter the page.
def count_equal(url):
    return url.count('=')

In [ ]:
#Length of URL
def url_length(url):
    return len(str(url))

In [ ]:
#Hostname Length
def hostname_length(url):
    return len(urlparse(url).netloc)

In [ ]:
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

In [ ]:
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

In [ ]:
#First Directory Length
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

In [23]:
def create_features(df):
    df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))
    df['abnormal_url'] = df['url'].apply(lambda i: abnormal_url(i))
    df['count.'] = df['url'].apply(lambda i: count_dot(i))
    df['count-www'] = df['url'].apply(lambda i: count_www(i))
    df['count@'] = df['url'].apply(lambda i: count_atrate(i))
    df['count_dir'] = df['url'].apply(lambda i: no_of_dir(i))
    df['count_embed_domian'] = df['url'].apply(lambda i: no_of_embed(i))
    df['count-https'] = df['url'].apply(lambda i : count_https(i))
    df['count-http'] = df['url'].apply(lambda i : count_http(i))
    df['count%'] = df['url'].apply(lambda i : count_per(i))
    df['count?'] = df['url'].apply(lambda i: count_ques(i))
    df['count-'] = df['url'].apply(lambda i: count_hyphen(i))
    df['count='] = df['url'].apply(lambda i: count_equal(i))
    df['url_length'] = df['url'].apply(lambda i: url_length(i))
    df['hostname_length'] = df['url'].apply(lambda i: hostname_length(i))
    df['count-digits']= df['url'].apply(lambda i: digit_count(i))
    df['count-letters']= df['url'].apply(lambda i: letter_count(i))
    df['fd_length'] = df['url'].apply(lambda i: fd_length(i))
    return df